<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [112]:
import pandas as pd
import tweepy
import re
import regex
import emoji

In [113]:
keys = pd.read_csv("../Clases/mis_llaves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [114]:
tweets = tweepy.Cursor(api.search,
                       q=['#Cúcuta -RT'],
                       lang='es',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

In [115]:
t = [tweet for tweet in tweets]

In [116]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Cúcuta')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    t_processed.append(temp)
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data

,tweets,date,city,user_location,id,link
0,Durante las actividades de verificación adelan...,2021-05-14 02:20:27,Cúcuta,Colombia,1393028405118001154,https://twitter.com/i/web/stuatus/139302840511...
1,#Málagavive #loúltimo | \n\nIglesias de Málaga...,2021-05-14 02:14:47,Cúcuta,Malaga santander,1393026978932998147,https://twitter.com/i/web/stuatus/139302697893...
2,🏡 Bosque de la variante\n🏡En el mes de las mad...,2021-05-14 01:57:00,Cúcuta,"Cúcuta, Norte de Santander",1393022501974941696,https://twitter.com/i/web/stuatus/139302250197...
3,#LasTresDelOriente ✅| Conozca las tres noticia...,2021-05-14 01:39:56,Cúcuta,Bucaramanga - Cúcuta,1393018208244879363,https://twitter.com/i/web/stuatus/139301820824...
4,"Mañana se estrenará la obra ""El #bicentenario ...",2021-05-14 01:28:05,Cúcuta,"Cúcuta, Colombia",1393015227822874625,https://twitter.com/i/web/stuatus/139301522782...
...,...,...,...,...,...,...
95,#EsNoticiaTRO🚨l Trabajo articulado entre alcal...,2021-05-13 14:13:30,Cúcuta,Bucaramanga - Cúcuta,1392845459899699202,https://twitter.com/i/web/stuatus/139284545989...
96,#FelizJueves ¿Cuáles son los tipos de tapaboca...,2021-05-13 14:11:19,Cúcuta,En 5 departamentos de Colombia,1392844911511212034,https://twitter.com/i/web/stuatus/139284491151...
97,🏡 Bosque de la variante\n🏡En el mes de las mad...,2021-05-13 13:56:20,Cúcuta,"Cúcuta, Norte de Santander",1392841143038132229,https://twitter.com/i/web/stuatus/139284114303...
98,"Buen Día Amigos, Busco Empleo en la Ciudad de ...",2021-05-13 13:42:07,Cúcuta,,1392837564684546048,https://twitter.com/i/web/stuatus/139283756468...


In [117]:
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

In [118]:
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  
data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()

,tweets,date,city,user_location,id,link,texto_limpio,links,hashtags,mentions,emojis
0,Durante las actividades de verificación adelan...,2021-05-14 02:20:27,Cúcuta,Colombia,1393028405118001154,https://twitter.com/i/web/stuatus/139302840511...,Durante las actividades de verificación adelan...,[https://t.co/usqBVWdqMY],"[#GEM,, #Cúcuta,, #EstatutoTemporaldeProtección.]","[@MigracionCol,]",[]
1,#Málagavive #loúltimo | \n\nIglesias de Málaga...,2021-05-14 02:14:47,Cúcuta,Malaga santander,1393026978932998147,https://twitter.com/i/web/stuatus/139302697893...,| \n\nIglesias de Málaga enviaron un mensaje...,[https://t.co/eSHIEqozYU],"[#Málagavive, #loúltimo, #Málagavive, #familia...",[],[]
2,🏡 Bosque de la variante\n🏡En el mes de las mad...,2021-05-14 01:57:00,Cúcuta,"Cúcuta, Norte de Santander",1393022501974941696,https://twitter.com/i/web/stuatus/139302250197...,🏡 Bosque de la variante\n🏡En el mes de las mad...,"[https://t.co/DHZ54OgkXw, https://t.co/z134vF5...","[#SiguemeYTeSigo, #Retwet, #Cúcuta, #Colombia,...",[],"[🏡, 🏡, 💰]"
3,#LasTresDelOriente ✅| Conozca las tres noticia...,2021-05-14 01:39:56,Cúcuta,Bucaramanga - Cúcuta,1393018208244879363,https://twitter.com/i/web/stuatus/139301820824...,✅| Conozca las tres noticias más destacadas d...,[https://t.co/AZrLHUy4B0],"[#LasTresDelOriente, #Jueves, #Santander., #Cú...",[],[✅]
4,"Mañana se estrenará la obra ""El #bicentenario ...",2021-05-14 01:28:05,Cúcuta,"Cúcuta, Colombia",1393015227822874625,https://twitter.com/i/web/stuatus/139301522782...,"Mañana se estrenará la obra ""El del congreso ...",[https://t.co/BxFUQ41OAm],"[#bicentenario, #Cúcuta"", #Teatro, #Cultura]",[],[🎭]


In [119]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp

,hora,cnt
77,00:14:00,1
29,00:26:00,1
62,01:28:00,1
46,01:39:00,1
84,01:57:00,1
...,...,...
22,22:44:00,1
74,23:13:00,1
93,23:35:00,1
17,23:36:00,1


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [120]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [121]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)